In [ ]:
import os
import sys
import numpy as np
if '..' not in sys.path:
    sys.path.append('..')
from pfcommon import parse_sparse_matrix_file, parse_Amat_vars_file, parse_Jacobian_vars_file

In [ ]:
def print_matrix(M, var_names, outfile=None):
    n_vars = len(var_names)
    n_char = max(max(map(len, var_names)), 8)
    fmt_str = ' {:>' + str(n_char) + 's}'
    fmt_int = ' {:' + str(n_char) + 'g}'
    fmt_num = ' {:' + str(n_char) + '.4f}'
    if outfile is not None:
        out = open(outfile,'w')
    else:
        out = sys.stdout
    out.write(' ' * 5)
    for var_name in var_names:
        out.write(fmt_str.format(var_name))
    out.write('\n')
    for i,row in enumerate(M):
        out.write(f'[{i+1:02d}] ')
        for val in row:
            if np.abs(val)<1e-6 or np.abs(1-np.abs(val))<1e-6:
                out.write(fmt_int.format(val))
            else:
                out.write(fmt_num.format(val))
        out.write('\n')
    if outfile is not None:
        out.close()

In [ ]:
model_name = 'SM_with_loads'
dynamic = True
load_type = 'const_S'
if not dynamic and load_type != 'const_Z':
    raise Exception('Load type must be "const_Z" if load is static')
expt_name = '{}_{}_{}'.format('dynamic' if dynamic else 'static',
                              'loads' if 'loads' in model_name else 'load',
                              load_type)
folder = os.path.join('..','..','modal_analysis',model_name,expt_name)
filename = os.path.join(folder, 'VariableToIdx_Jacobian.txt')
vars_idx,state_vars,voltages,currents,signals = parse_Jacobian_vars_file(filename)
filename = os.path.join(folder, 'Jacobian.mtl')
J = parse_sparse_matrix_file(filename, )
filename = os.path.join(folder, model_name + '_AC.npz')
data = np.load(filename, allow_pickle=True)
S = data['S'].item()
PF = data['PF_without_slack'].item()
PF_bus = PF['buses']['Bus1']
PF_loads = PF['loads']
PF_gen = PF['SMs']['G1']
cosphi = PF_gen['cosphi']
ϕ = np.arccos(cosphi)
print('cos(ϕ) = {:7.3f}'.format(cosphi))
print('     ϕ = {:7.3f} deg'.format(np.rad2deg(ϕ)))

In [ ]:
flatten = lambda D: [k+'.'+el for k,subl in D.items() for el in subl]
var_names = flatten(state_vars) + flatten(voltages) + flatten(currents)
n_vars = len(var_names)
print('State variables: "{}".'.format('", "'.join(flatten(state_vars))))
print('       Voltages: "{}".'.format('", "'.join(flatten(voltages))))
print('       Currents: "{}".'.format('", "'.join(flatten(currents))))

#### Base parameters
These values are used by PowerFactory for display purposes

In [ ]:
F      = 50.  # [Hz] default frequency
S_base = 1e6  # [VA] base apparent power
V_base = PF_bus['Vl']/PF_bus['u']*1e3  # [V] base voltage (line-to-line)
I_base = S_base/V_base                 # [A] base current
Z_base = V_base**2/S_base              # [Ω] base impedance
Y_base = 1/Z_base                      # [S] base admittance
print('====== System ======')
print('S_base = {:7.3f} MVA'.format(S_base*1e-6))
print('V_base = {:7.3f} kV'.format(V_base*1e-3))
print('I_base = {:7.3f} kA'.format(I_base*1e-3))
print('Z_base = {:7.3f} Ω'.format(Z_base))
print('Y_base = {:7.3f} S'.format(Y_base))

#### Generator parameters
Base parameters of the single generator in the network

In [ ]:
S_base_gen = S['G1']*1e6                      # [VA]
V_base_gen = PF_gen['Vl']*1e3 / PF_gen['u']   # [V]
I_base_gen = S_base_gen/V_base_gen            # [A]
Z_base_gen = V_base_gen**2 / S_base_gen       # [Ω]
Y_base_gen = 1 / Z_base_gen                   # [S]
rstr,xstr = 0.2, 0.4                          # [pu] stator parameters
R_gen,X_gen = rstr*Z_base_gen,xstr*Z_base_gen # [Ω]
Z_gen = R_gen + 1j*X_gen                      # [Ω]
Y_gen = 1/Z_gen                               # [S]
Z_gen_pu = Z_gen/Z_base                       # [pu]
Y_gen_pu = Y_gen/Y_base                       # [pu]
R_gen_pu,X_gen_pu = Z_gen_pu.real, Z_gen_pu.imag
gen_coeffs = np.array([[R_gen_pu, -X_gen_pu],[X_gen_pu, R_gen_pu]])

V = (Z_gen/Z_base_gen) * (PF_gen['ir'] + 1j*PF_gen['ii'])
e = (PF_bus['ur'] + 1j*PF_bus['ui']) + V
E0,ϕg = np.abs(e), np.angle(e)
phase_coeffs = np.array([E0*np.sin(ϕg), -E0*np.cos(ϕg)])

print('===== Generator ====')
print('S_base = {:7.3f} MVA'.format(S_base_gen*1e-6))
print('V_base = {:7.3f} kV'.format(V_base_gen*1e-3))
print('I_base = {:7.3f} kA'.format(I_base_gen*1e-3))
print('Z_base = {:7.3f} Ω'.format(Z_base_gen))
print('Y_base = {:7.3f} S'.format(Y_base_gen))

#### Load parameters
The p.u. values of the load impedance are referred to the system base.

In [ ]:
load_names = [k for k,v in PF_loads.items() if isinstance(v,dict)]
load_types = {name: load_type for name in load_names}
load_coeffs = {name: np.zeros((2,2)) for name in load_names}
for load_name in load_names:
    PF_load = PF_loads[load_name]
    ld_typ = load_types[load_name]
    if ld_typ == 'const_Z':
        u = np.sqrt(3) * PF_load['V'] * np.exp(1j*np.deg2rad(PF_load['phiu'])) # [kV]
        i = np.sqrt(3) * PF_load['I'] * np.exp(1j*np.deg2rad(PF_load['phii'])) # [kA]
        S = u*i.conjugate() # [MVA]
        assert np.abs(S.real - PF_load['P']) < 1e-6
        assert np.abs(S.imag - PF_load['Q']) < 1e-6
        Y_load    = i/u                                      # [S]
        Y_load_pu = Y_load/Y_base                            # [pu]
        G_load,B_load       = Y_load.real, Y_load.imag       # [S]
        G_load_pu,B_load_pu = Y_load_pu.real, Y_load_pu.imag # [pu]
        load_coeffs[load_name] = np.array([[G_load_pu,-B_load_pu],[B_load_pu,G_load_pu]])
        if dynamic:
            ##############################################
            ### DON'T KNOW WHY THIS WORKS/IS NECESSARY ###
            ##############################################
            load_coeffs[load_name][0,0] = G_load_pu-B_load_pu
            load_coeffs[load_name][1,0] = (1-B_load_pu/G_load_pu) * B_load_pu
        print('========== {} ========='.format(load_name))
        print('G = {:7.5f} S, {:5.3f} pu'.format(G_load, G_load_pu))
        print('B = {:7.5f} S, {:5.3f} pu'.format(B_load, B_load_pu))
    elif ld_typ == 'const_S':
        P = PF_load['P']*1e6/S_base  # [pu]
        Q = PF_load['Q']*1e6/S_base  # [pu]
        ur,ui = PF_bus['ur'], PF_bus['ui']
        den = (ur**2+ui**2)**2       # [pu]
        load_coeffs[load_name] = np.array([[(P*(ui**2-ur**2) - 2*Q*ur*ui) / den, (Q*(ur**2-ui**2) - 2*P*ur*ui) / den],
                                           [(Q*(ur**2-ui**2) - 2*P*ur*ui) / den, (P*(ur**2-ui**2) + 2*Q*ur*ui) / den]])
        print('======== {} ========'.format(load_name))
        print('S = {:g} MVA = {:g} pu'.format(PF_load['P']+1j*PF_load['Q'], P+1j*Q))
    else:
        raise Exception('Unknown load type `{ld_typ}`')

## Power flow results
#### Generator

In [ ]:
print('     P = {:7.3f} MW'.format(PF_gen['P']))
print('     Q = {:7.3f} Mvar'.format(PF_gen['Q']))
print('     u = {:7.3f} pu'.format(PF_gen['u']))
print('    ur = {:7.3f} pu'.format(PF_gen['ur']))
print('    ui = {:7.3f} pu'.format(PF_gen['ui']))
print(' V_l2l = {:7.3f} kV'.format(PF_gen['Vl']))
print(' V_l2g = {:7.3f} kV'.format(PF_gen['V']))
print('   V_ϕ = {:7.3f} deg'.format(PF_gen['phiu']))
print('     I = {:7.3f} kA'.format(PF_gen['I']))
print('     i = {:7.3f} pu'.format(PF_gen['i']))
print('    ir = {:7.3f} pu'.format(PF_gen['ir']))
print('    ii = {:7.3f} pu'.format(PF_gen['ii']))

#### Load(s)

In [ ]:
for load_name in load_names:
    PF_load = PF_loads[load_name]
    print('>>> Load {}'.format(load_name))
    print('     P = {:7.3f} MW'.format(PF_load['P']))
    print('     Q = {:7.3f} Mvar'.format(PF_load['Q']))
    print('     u = {:7.3f} pu'.format(PF_load['u']))
    print('    ur = {:7.3f} pu'.format(PF_load['ur']))
    print('    ui = {:7.3f} pu'.format(PF_load['ui']))
    print(' V_l2l = {:7.3f} kV'.format(PF_load['Vl']))
    print(' V_l2g = {:7.3f} kV'.format(PF_load['V']))
    print('   V_ϕ = {:7.3f} deg'.format(PF_load['phiu']))
    print('     I = {:7.3f} kA'.format(PF_load['I']))
    print('     i = {:7.3f} pu'.format(PF_load['i']))
    print('    ir = {:7.3f} pu'.format(PF_load['ir']))
    print('    ii = {:7.3f} pu'.format(PF_load['ii']))

#### Bus

In [ ]:
print('     u = {:7.3f} pu'.format(PF_bus['u']))
print('    ur = {:7.3f} pu'.format(PF_bus['ur']))
print('    ui = {:7.3f} pu'.format(PF_bus['ui']))
print(' V_l2l = {:7.3f} kV'.format(PF_bus['Vl']))
print(' V_l2g = {:7.3f} kV'.format(PF_bus['V']))
print('     ϕ = {:7.3f} deg'.format(PF_bus['phi']))

### Static load
  1. A static load is represented as a constant impedance.
  1. The number of variables is equal to 6.
  1. The submatrix has the following structure, where all the values are in per unit:

|Variable|$\frac{\partial}{\partial\phi}$|$\frac{\partial}{\partial u_r}$|$\frac{\partial}{\partial u_i}$|$\frac{\partial}{\partial i_r^G}$|$\frac{\partial}{\partial i_i^G}$|
|:---:|:---:|:---:|:---:|:---:|:---:|
|**$u_r$**| 0 | $G_L$ | $-B_L$ | -1 |  0 |
|**$u_i$**| 0 | $B_L$ | $G_L$ |  0 | -1 |
|**$i_r^G$**| $E_0\sin(\phi)$ | 1 | 0 | $R_G$ | $-X_G$ |
|**$i_i^G$**| $E_0\cos(\phi)$ | 0 | 1 | $X_G$ | $R_G$ |

where $(u_r,u_i)$ are the real and imaginary parts of the voltage at the bus, $(i_r^G,i_i^G)$ are the real and imaginary parts of the generator current, $(R_G,X_G)$ are the resistance and reactance of the synchronous machine's stator, and $(G_L,B_L)$ are the conductance and susceptance of the load.

### Dynamic load
  1. Dynamic load with constant power.
  1. The number of variables is equal to 8.
  1. The submatrix has the following structure, where all the values are in per unit:

|Variable|$\frac{\partial}{\partial\phi}$|$\frac{\partial}{\partial u_r}$|$\frac{\partial}{\partial u_i}$|$\frac{\partial}{\partial i_r^L}$|$\frac{\partial}{\partial i_i^L}$|$\frac{\partial}{\partial i_r^G}$|$\frac{\partial}{\partial i_i^G}$|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|**$u_r$**| 0 | 0 | 0 | 1 | 0 | -1 |  0 |
|**$u_i$**| 0 | 0 | 0 | 0 | 1 |  0 | -1 |
|**$i_r^G$**| 0 | $c_{11}$ | $c_{12}$ | -1 |  0 | 0 | 0 |
|**$i_i^G$**| 0 | $c_{21}$ | $c_{22}$ |  0 | -1 | 0 | 0 |
|**$i_r^L$**|  $E_0 \sin(\phi)$ | 1 | 0 | 0 | 0 | $R_G$ | $-X_G$ |
|**$i_i^L$**| $-E_0 \cos(\phi)$ | 0 | 1 | 0 | 0 | $X_g$ |  $R_G$ |

where $(u_r,u_i)$ are the real and imaginary parts of the voltage at the bus, $(i_r^G,i_i^G)$ are the real and imaginary parts of the generator current, $(i_r^L,i_i^L)$ are the real and imaginary parts of the load current, and $(R_G,X_G)$ are the resistance and reactance of the synchronous machine's stator.

The coefficients $c_{11}$, $c_{12}$, $c_{21}$ and $c_{22}$ are given by the following expressions:

$c_{11} = \frac{\partial}{\partial u_r} \frac{P u_r + Q u_i}{u_r^2 + u_i^2} = \frac{P(u_i^2-u_r^2) - 2Q u_r u_i}{(u_r^2+u_i^2)^2}$

$c_{12} = \frac{\partial}{\partial u_i} \frac{P u_r + Q u_i}{u_r^2 + u_i^2} = \frac{Q(u_r^2-u_i^2) - 2P u_r u_i}{(u_r^2+u_i^2)^2}$

$c_{21} = \frac{\partial}{\partial u_r} \frac{-Q u_r + P u_i}{u_r^2 + u_i^2} = \frac{Q(u_r^2-u_i^2) - 2P u_r u_i}{(u_r^2+u_i^2)^2}$

$c_{22} = \frac{\partial}{\partial u_i} \frac{-Q u_r + P u_i}{u_r^2 + u_i^2} = \frac{P(u_r^2-u_i^2) + 2Q u_r u_i}{(u_r^2+u_i^2)^2}$

In [ ]:
IDX = lambda name: var_names.index(name)
J_guess = np.zeros((n_vars,n_vars), dtype=float)

In [ ]:
if n_vars in (6,8,10):
    for i,ki in enumerate('ri'):
        idx = IDX('Bus1.u' + ki)
        for load_name in load_names:
            try:
                jdx = IDX(load_name + '.i' + ki)
                J_guess[idx,jdx] = 1
            except:
                for j,kj in enumerate('ri'):
                    jdx = IDX('Bus1.u' + kj)
                    J_guess[idx,jdx] += load_coeffs[load_name][i,j]
        jdx = IDX('G1.i' + ki)
        J_guess[idx,jdx] = -1

    for load_name in load_names:
        for i,ki in enumerate('ri'):
            try:
                idx = IDX(load_name + '.i' + ki)
                for j,kj in enumerate('ri'):
                    jdx = IDX('Bus1.u'+ kj)
                    J_guess[idx,jdx] = load_coeffs[load_name][i,j]
                J_guess[idx,idx] = -1
            except:
                pass

    for i,ki in enumerate('ri'):
        idx = IDX('G1.i' + ki)
        jdx = IDX('G1.phi')
        J_guess[idx,jdx] = phase_coeffs[i]
        jdx = IDX('Bus1.u' + ki)
        J_guess[idx,jdx] = 1
        for j,kj in enumerate('ri'):
            jdx = IDX('G1.i' + kj)
            J_guess[idx,jdx] = gen_coeffs[i,j]
else:
    raise Exception(f'Do not know how to deal with {n_vars} variables')

In [ ]:
print_matrix(J[:n_vars,:n_vars], var_names, 'J.out' if n_vars > 10 else None)

In [ ]:
print_matrix(J_guess, var_names, 'J_guess.out' if n_vars > 10 else None)